In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00


In [3]:
%pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 20.8 MB/s eta 0:00:00


In [ ]:
# --- RUN THIS CELL TO FIX THE ERROR ---

# 1. Force-uninstall any broken versions
print("Uninstalling old versions...")
!pip uninstall -y torch torchaudio torchdata torchvision fastai

# 2. Reinstall the correct, matching versions
print("\nInstalling fresh, matching versions...")
!pip install torch torchaudio torchvision fastai torchcodec

# 3. Restart runtime to apply changes
print("\nRestarting runtime...")
import os
os.kill(os.getpid(), 9)

Uninstalling old versions...
Found existing installation: torch 2.9.0
Uninstalling torch-2.9.0:
  Successfully uninstalled torch-2.9.0
Found existing installation: torchaudio 2.9.0
Uninstalling torchaudio-2.9.0:
  Successfully uninstalled torchaudio-2.9.0
ERROR: Operation cancelled by user

Installing fresh, matching versions...
  Using cached torch-2.9.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached torchaudio-2.9.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (6.9 kB)
Using cached torch-2.9.0-cp312-cp312-manylinux_2_28_x86_64.whl (899.7 MB)
Using cached torchaudio-2.9.0-cp312-cp312-manylinux_2_28_x86_64.whl (2.1 MB)


In [ ]:
!pip install librosa pandas

In [9]:
%pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=d695f3d02de3bcc23ec5fce31b9cf4d5ff7aba4bb5b201853055b338475b7ad7
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [12]:
# This script performs evaluation of OpenAI's Whisper model on your dataset.
# It does the following:
# 1. Loads your CSV file with audio filenames and their transcripts.
# 2. Loads a pre-trained Whisper model.
# 3. Iterates through each audio file, transcribes it using Whisper.
# 4. Compares the 'predicted' text from Whisper to the 'actual' text from your CSV.
# 5. Calculates and reports the overall Word Error Rate (WER).
#
# --- Installation ---
# You must install the required libraries:
# pip install openai-whisper pandas jiwer
#
# --- IMPORTANT: FFMPEG Requirement ---
# Whisper requires a system-level tool called 'ffmpeg' to read MP3s.
# - On Ubuntu/Debian: sudo apt update && sudo apt install ffmpeg
# - On macOS (using Homebrew): brew install ffmpeg
# - On Windows (using Chocolatey): choco install ffmpeg
#   (Or download the ffmpeg.exe and add it to your system's PATH)

import os
import whisper
import pandas as pd
import jiwer  # Library for calculating Word Error Rate
from tqdm import tqdm  # For a nice progress bar
import zipfile # Import zipfile

# --- 1. CONFIGURE YOUR PATHS HERE ---

# -----------------------------------------------------------------------------
# *** This is where you feed your files ***
# -----------------------------------------------------------------------------

# Path to your CSV file (e.g., "C:/Users/YourUser/Desktop/my_data/metadata.csv")
CSV_FILE_PATH = "/content/VCD-G2.csv"

# Path to the ZIP file containing all your audio files
AUDIO_ZIP_PATH = "/content/Audios.zip"

# Temporary directory to extract audio files
TEMP_EXTRACT_DIR = "/content/temp_audio_extracted_whisper"

# This will be the actual directory containing the audio files after extraction
# AUDIO_FOLDER_PATH = None # This will be set dynamically - Removed

# -----------------------------------------------------------------------------
# *** Configure your CSV column names ***
# -----------------------------------------------------------------------------

# The name of the column in your CSV that contains the audio filename (e.g., "file1.mp3")
AUDIO_FILENAME_COLUMN = "LINK_1"

# The name of the column in your CSV that contains the correct transcript
TRANSCRIPTION_COLUMN = "TRANSCRIPT"

# -----------------------------------------------------------------------------
# *** Configure the Whisper Model ***
# -----------------------------------------------------------------------------
# Model size. Options: "tiny", "base", "small", "medium", "large"
# "base" or "small" are good starting points. "large" is most accurate but slowest.
# Add ".en" (e.g., "base.en") if you know your audio is only in English.
MODEL_SIZE = "small.en"
# -----------------------------------------------------------------------------


def load_data(csv_path):
    """Loads the metadata CSV into a pandas DataFrame."""
    print(f"Loading metadata from {csv_path}...")
    try:
        df = pd.read_csv(csv_path)
    except FileNotFoundError:
        print(f"ERROR: CSV file not found at: {csv_path}")
        print("Please update the 'CSV_FILE_PATH' variable in the script.")
        return None
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return None

    # Check for required columns
    if AUDIO_FILENAME_COLUMN not in df.columns or TRANSCRIPTION_COLUMN not in df.columns:
        print(f"ERROR: CSV must contain '{AUDIO_FILENAME_COLUMN}' and '{TRANSCRIPTION_COLUMN}' columns.")
        print("Please update the column name variables in the script if yours are different.")
        return None

    print(f"Found {len(df)} entries in {METADATA_CSV_PATH}")
    return df

def preprocess_text(text):
    """
    Applies standard text cleaning to both reference and predicted text
    to ensure a fair comparison for WER calculation.
    """
    if not isinstance(text, str):
        text = str(text)

    text = text.lower()  # Convert to lowercase
    # This is jiwer's standard transformation
    text = jiwer.RemoveMultipleSpaces()(text)
    text = jiwer.RemovePunctuation()(text)
    text = text.strip()
    return text

# Modified evaluate_dataset to search the whole directory
def evaluate_dataset(df, extracted_dir, model):
    """
    Transcribes all audio files and compares them to the reference transcripts.
    Searches the entire extracted directory for each file.
    """
    predictions = []
    references = []
    processed_count = 0

    print(f"\nStarting transcription with '{MODEL_SIZE}' model...")

    # Use tqdm for a progress bar
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Transcribing"):
        try:
            audio_filename = row[AUDIO_FILENAME_COLUMN]
            reference_transcript = row[TRANSCRIPTION_COLUMN]

            # Check if file_name is a valid string and not NaN
            if not isinstance(audio_filename, str) or pd.isna(audio_filename):
                # print(f"\nSkipping row {index} due to invalid file name: {audio_filename}")
                continue

            found_file_path = None
            # Search for the file in the entire extracted directory structure
            for root, dirs, files in os.walk(extracted_dir):
                if audio_filename in files:
                    found_file_path = os.path.join(root, audio_filename)
                    break # Stop searching once the file is found

            if not found_file_path:
                # print(f"\nWarning: File not found in extracted directory, skipping: {audio_filename}")
                continue

            # 2. Preprocessing & Test (Transcription)
            # Whisper handles the audio loading and preprocessing internally.
            # Using fp16=False is more compatible with CPU-only execution.
            # If you have a powerful NVIDIA GPU, you can remove fp16=False.
            result = model.transcribe(found_file_path, fp16=False)
            predicted_transcript = result['text']

            # 3. Store cleaned results for comparison
            predictions.append(preprocess_text(predicted_transcript))
            references.append(preprocess_text(reference_transcript))
            processed_count += 1

        except Exception as e:
            print(f"\nError processing file {audio_filename}: {e}")

    print(f"\nSuccessfully processed {processed_count} audio files.")
    return references, predictions

def main():
    """Main function to run the evaluation."""

    # 1. Data Extraction (Load CSV)
    df = load_data(CSV_FILE_PATH)
    if df is None:
        return

    # 2. Extract Audio to Temporary Directory
    print(f"\n--- Extracting Audio ---")
    try:
        with zipfile.ZipFile(AUDIO_ZIP_PATH, "r") as zip_ref:
            print(f"Extracting '{AUDIO_ZIP_PATH}' to '{TEMP_EXTRACT_DIR}'...")
            zip_ref.extractall(TEMP_EXTRACT_DIR)
        print("✅ Audio extraction complete.")

    except FileNotFoundError:
        print(f"ERROR: ZIP file not found at: {AUDIO_ZIP_PATH}")
        print("Please update the 'AUDIO_ZIP_PATH' variable and ensure the file exists.")
        return
    except Exception as e:
        print(f"Error extracting zip: {e}")
        return

    # Load Whisper Model
    try:
        print(f"\nLoading Whisper model '{MODEL_SIZE}'... (This may take a moment)")
        # Use device="cuda" if a GPU is available for faster processing
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model = whisper.load_model(MODEL_SIZE, device=device)
        print(f"✅ Whisper model loaded on {device}.")
    except Exception as e:
        print(f"Error loading Whisper model: {e}")
        print("Please ensure you have a valid model name and internet connection.")
        print("Also, check if ffmpeg is installed on your system.")
        return

    # 3. Run Evaluation
    # Pass the temporary extracted directory to the evaluation function
    references, predictions = evaluate_dataset(df, TEMP_EXTRACT_DIR, model)

    if not references or not predictions:
        print("\nNo files were processed. Exiting.")
        return

    # 4. Show Predicted vs. Actual
    print("\n--- Transcription Results (Actual vs. Predicted) ---")
    # We'll just show the first 10 results to avoid flooding the console
    for i in range(min(len(references), 10)):
        print(f"\nSample {i+1}:")
        print(f"  Actual:   {references[i]}")
        print(f"  Predicted: {predictions[i]}")
    if len(references) > 10:
        print(f"\n... (and {len(references) - 10} more results) ...")

    # 5. Calculate and Show WER
    print("\n--- Overall Performance ---")
    try:
        # Calculate Word Error Rate
        wer = jiwer.wer(references, predictions)
        print(f"Overall Word Error Rate (WER): {wer * 100:.2f} %")
        print(f"This means, on average, {wer*100:.2f}% of the words were incorrect.")
    except ValueError as e:
        print(f"Error calculating WER: {e}")

if __name__ == "__main__":
    main()

Loading metadata from /content/VCD-G2.csv...
Found 451 entries in /content/VCD-G2.csv

--- Extracting Audio ---
Extracting '/content/Audios.zip' to '/content/temp_audio_extracted_whisper'...
✅ Audio extraction complete.

Loading Whisper model 'small.en'... (This may take a moment)
✅ Whisper model loaded on cpu.

Starting transcription with 'small.en' model...


Transcribing: 100%|██████████| 451/451 [1:11:46<00:00,  9.55s/it]


Successfully processed 302 audio files.

--- Transcription Results (Actual vs. Predicted) ---

Sample 1:
  Actual:   alright get the drone airborne take off
  Predicted: lets start the mission the curve on the mark

Sample 2:
  Actual:   lets start the mission take off on my mark
  Predicted: drone lift and take off

Sample 3:
  Actual:   power up the rotors and take off
  Predicted: dont take off 2 meters

Sample 4:
  Actual:   initiate take off
  Predicted: dont take off straight up

Sample 5:
  Actual:   lets see what you can do take off
  Predicted: lets see what you can do take off

Sample 6:
  Actual:   bring it home land in the designated zone
  Predicted: john please learn

Sample 7:
  Actual:   okay thats a wrap lets land
  Predicted: june land immediately

Sample 8:
  Actual:   were done here execute landing procedure land
  Predicted: can you land

Sample 9:
  Actual:   the battery is low land the drone
  Predicted: its time to learn

Sample 10:
  Actual:   set it down gent